# Main CNN model for bat call classification

### **TO RUN THIS FILE**, just press the "restart&execute" button, and when the kernel dies, restart the notebook process, go to the `exit()` cell and execute all cells after that one

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow_addons.metrics import F1Score
from keras import backend as K 
import cv2
import time
from sklearn.model_selection import train_test_split
import itertools_len as itertools
from itertools_len import product
import gc
from tensorflow.keras.optimizers.legacy import Adam, SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTEENN, SMOTETomek
from tensorflow.keras import regularizers
import tensorflow_model_optimization as tfmot
import tempfile
import os

2023-12-25 19:26:11.076583: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-25 19:26:11.175407: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-25 19:26:11.175432: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-12-25 19:26:11.653859: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
data = pd.read_pickle('./data/images_df_numerical.pkl')

def split_df_equal_class_distribution(df, batch_size):
    
    df['temp_id'] = range(len(df))
    
    num_batches = int(np.ceil(len(df) / batch_size))
    
    grouped = df.groupby('Species', group_keys=False)
    
    chunks = []
    
    for i in range(num_batches):
        chunk = pd.DataFrame(columns=df.columns)
        for _, group in grouped:
            num_samples = int(batch_size * len(group) / len(df))
            sample_indices = np.random.choice(group['temp_id'], size=num_samples, replace=False)
            chunk = pd.concat([chunk, df[df['temp_id'].isin(sample_indices)]])
        chunk = chunk.drop('temp_id', axis=1)
        chunks.append(chunk)
    
    return chunks

chunk_size = 1000

chunks_with_same_dist = split_df_equal_class_distribution(data, chunk_size)
del data
classes = chunks_with_same_dist[0]["Species"].unique()
x_len = chunks_with_same_dist[0].iloc[0]["data"].size
number_of_classes = classes.size
most_x_in_one_class = chunks_with_same_dist[0]["Species"].value_counts().iloc[0]

In [3]:
# Alleiniges undersampling wird keinen Sinn machen, da wir extrem wenig Datenpunkte overall haben
def resample(resampler) -> tuple[np.array, np.array]:
    # 0.3 as buffer
    array_size = int(most_x_in_one_class * number_of_classes * (len(chunks_with_same_dist) + 0.3))
    X = np.empty((array_size, x_len), dtype=np.uint8)
    y = np.empty((array_size), dtype=np.uint8)

    current_index = 0
    for chunk in chunks_with_same_dist:
        X_batch, y_batch = chunk['data'], chunk['Species']
        X_batch, y_batch = np.stack(X_batch).astype(np.uint8), y_batch.astype(np.uint8)
        X_resampled, y_resampled = resampler.fit_resample(X_batch, y_batch)
        num_samples = X_resampled.shape[0]
        X[current_index:current_index + num_samples] = X_resampled.astype(np.uint8)
        y[current_index:current_index + num_samples] = y_resampled.astype(np.uint8)
        current_index += num_samples

    X.resize((current_index, X.shape[1]))
    y.resize(current_index)
    print(f"{resampler}: ", pd.Series(y, dtype=pd.UInt8Dtype()).value_counts())

    return X, y

# oversampling
smote = SMOTE()
adasyn = ADASYN()

X, y = resample(adasyn)

# Kombination aus over und undersampling
smoteenn = SMOTEENN()
smotettomek = SMOTETomek()

ADASYN():  3    2302
4    2270
2    2261
0    2260
5    2256
1    2235
dtype: Int64


In [4]:
image_size = X[0].size
samples = X.size
image_shape = (65, 100, 3) # height, width , color channel
smallest_float16 = np.finfo(np.float16).tiny
# normalize to 0-1
X = X / 255.
X = X.reshape((-1,) + image_shape)
X = X.astype(smallest_float16)

In [5]:
with open("X.npy", "wb") as file:
    np.save(file, X)
with open("y.npy", "wb") as file:
    np.save(file, y)

In [6]:
kfold = KFold(n_splits=10, shuffle=True)

tf.keras.utils.set_random_seed(1)

# If using TensorFlow, this will make GPU ops as deterministic as possible,
# but it will affect the overall performance, so be mindful of that.
tf.config.experimental.enable_op_determinism()

In [7]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=30, min_delta=0.001, start_from_epoch=15, restore_best_weights=True)
epochs = 200
batch_size = 32
dropout_rate = 0.4 # https://www.kaggle.com/code/rafjaa/dealing-with-very-small-datasets interessant bzgl oberfitting
weight_decay_alpha = 0.01

def create_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=image_shape))
    model.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(number_of_classes, activation='softmax'))

    return model

def create_optimizers(num_samples) -> dict:
    s = 130 * num_samples // 32 # number of steps in 130 epochs (batch size = 32)
    exp_decay_sgd = ExponentialDecay(0.01, s, 0.1)
    exp_adam = ExponentialDecay(0.01, s, 0.95, staircase=True)

    momentum = 0.99
    sgd_exp = SGD(exp_decay_sgd, momentum=momentum)
    adam_exp = Adam(exp_adam)

    sgd = SGD(0.001, momentum=momentum)
    adam = Adam(0.001)

    return {"sgd_exp": sgd_exp, "adam_exp": adam_exp, "sgd": sgd, "adam": adam}

histories_with_params = list()

end_step = np.ceil(X.shape[0] / batch_size).astype(np.int32) * epochs

pruning_params = {
    # In this example, you start the model with 50% sparsity (50% zeros in weights) and end with 80% sparsity.
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                                final_sparsity=0.80,
                                                                begin_step=0,
                                                                end_step=end_step)}

In [8]:
def fit_model(model, X_train, y_train, worker=8):
    history = model.fit(
        X_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        workers=worker, # workers are number of cores
        callbacks=[early_stopping, tfmot.sparsity.keras.UpdatePruningStep()],
        validation_split=0.2,
        verbose=1)

    return history

In [10]:
chunk_size = 1000
y = np.load("y.npy", mmap_mode="r+")
num_samples = len(y)
del y
gc.collect()

model = create_model()

optimizers = create_optimizers(num_samples)
for optimizer_name, optimizer in optimizers.items():
    model = tfmot.sparsity.keras.prune_low_magnitude(model, **pruning_params)
    for i in range(0, num_samples, chunk_size):
        X = np.load("X.npy", mmap_mode="r+")
        y = np.load("y.npy", mmap_mode="r+")
        for train_indezes, test_indezes in kfold.split(X[i:i+chunk_size], y[i:i+chunk_size]):
            X_train, y_train = tf.convert_to_tensor(X[train_indezes]), tf.convert_to_tensor(y[train_indezes])
            X_test, y_test = tf.convert_to_tensor(X[train_indezes]), tf.convert_to_tensor(y[test_indezes])
            model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
            K.clear_session()
            history = fit_model(model, X_train, y_train)

Epoch 1/200
23/23 [==============================] - 2s 61ms/step - loss: 11.4702 - accuracy: 0.3500 - val_loss: 99.8486 - val_accuracy: 0.0278
Epoch 2/200
23/23 [==============================] - 1s 52ms/step - loss: 17.2586 - accuracy: 0.6458 - val_loss: 269.3952 - val_accuracy: 0.0167
Epoch 3/200
23/23 [==============================] - 1s 53ms/step - loss: 26.2496 - accuracy: 0.6069 - val_loss: 105.5293 - val_accuracy: 0.0167
Epoch 4/200
23/23 [==============================] - 1s 55ms/step - loss: 31.9442 - accuracy: 0.6125 - val_loss: 51.4174 - val_accuracy: 0.3444
Epoch 5/200
23/23 [==============================] - 1s 54ms/step - loss: 31.9176 - accuracy: 0.6472 - val_loss: 39.2450 - val_accuracy: 0.0556
Epoch 6/200
23/23 [==============================] - 1s 54ms/step - loss: 27.0985 - accuracy: 0.6347 - val_loss: 32.4499 - val_accuracy: 0.0056
Epoch 7/200
23/23 [==============================] - 1s 57ms/step - loss: 19.7177 - accuracy: 0.6139 - val_loss: 25.0297 - val_accurac

KeyboardInterrupt: 

In [ ]:
# Training and validating the model using KFold
history_with_param = {"optimizer": optimizer_name, "history": history}
histories_with_params.append(history_with_param)

In [ ]:
number_of_epochs = len(history.history["accuracy"])
for history_with_param in histories_with_params:
    K.clear_session()
    model = load_model(f"cnn_files/model_{history_with_param['optimizer']}.keras")
    test_score = round(model.evaluate(X_test, y_test)[1], 2)*100
    gc.collect()

    plt.figure()
    plt.plot(history_with_param["history"].history["accuracy"], label="train_data accuracy")
    plt.plot(history_with_param["history"].history["val_accuracy"], label="val_data accuracy")
    plt.scatter(number_of_epochs, test_score/100, label="test_data accuracy", marker="x", c="g")
    plt.title(f"opt: {history_with_param['optimizer']} Test Score: {test_score}%")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend(loc="upper left")
    plt.savefig(f"./cnn_files/{history_with_param['optimizer']}.png",dpi=600)
    #plt.show()